# Indexing

Alle Daten in die Datenbank bringen. Ich verwende ChromaDB weil lokal. Es ist die Grundlage für das spätere System und gleichzeitig die Quelle für die Evaluation.

In [2]:
import chromadb
import random
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer


/home/bruce/.local/share/mise/installs/python/3.13.7/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Embeddings laden

In [3]:
embeddings = np.load('../2-embedding/embeddings_gbert.npy')
chunks_meta = pd.read_json('../2-embedding/chunks_metadata.jsonl', lines=True)

## DB und Collection erstellen

In [8]:
client = chromadb.PersistentClient(path="./chroma_db")

client.delete_collection('prdukt_chunks')

collection = client.create_collection(
    name = "prdukt_chunks",
    metadata = {"description": "Product chunks with GBERT embeddings"}
)

## Daten vorverarbeiten

In [5]:
ids = [str(i) for i in range(len(chunks_meta))]
documents = chunks_meta['document'].tolist()
metadatas = chunks_meta['metadata'].tolist()
embeddings_list = embeddings.tolist()

## Indexieren

In [7]:
collection.add(
    ids = ids,
    documents = documents,
    embeddings = embeddings_list,
    metadatas = metadatas
)

print(f"Done")

Done


## Check DB

In [9]:
print(f"Anzahl der Dokumente: {collection.count()}")

sample_idx = [str(i) for i in random.sample(range(len(embeddings)), int(len(embeddings) * 0.005))]

result = collection.get(
    ids = sample_idx,
    include = ["documents", "metadatas", "embeddings"]
)

for i, (doc, meta) in enumerate(zip(result['documents'], result["metadatas"])):
    print(f"--- Dokuemnt {i} ---")
    print(f"Text: {doc[:200]}")
    print(f"Metadata: {meta}")

Anzahl der Dokumente: 0
